In [1]:
#!pip install pyomo
#!pip install cplex
#!pip install ipopt
#!pip install glpk, cbc

#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org glpk

In [2]:
import pyomo.environ as pyEnv

In [3]:
lucros = [92, 57, 49, 68, 60, 43, 67, 84, 87, 72]
pesos =  [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]
capacidade = 165
m = len(lucros)

In [4]:
modelo = pyEnv.ConcreteModel()
modelo.I = pyEnv.RangeSet(m)

modelo.x = pyEnv.Var(modelo.I, within=pyEnv.Binary)
modelo.obj = pyEnv.Objective(expr = sum(lucros[i-1]*modelo.x[i] for i in modelo.I), sense=pyEnv.maximize)
modelo.res = pyEnv.Constraint(expr = sum(pesos[i-1]*modelo.x[i] for i in modelo.I) <= capacidade)

In [5]:
path = '../solvers/cbc.exe'

In [6]:
#path = 'C:\Users\CarlosSolano.iel\Videos\DataAnalytics\optimizacao_linear'

pt = pyEnv.SolverFactory('cbc', executable=path)
#opt = pyEnv.SolverFactory('glpk', executable=path)
res_obj = pt.solve(modelo, tee=False)

In [7]:
lista =  list(modelo.x.keys())

for i in lista:
    print(i, '----', modelo.x[i]())

1 ---- 1.0
2 ---- 1.0
3 ---- 1.0
4 ---- 1.0
5 ---- 0.0
6 ---- 1.0
7 ---- 0.0
8 ---- 0.0
9 ---- 0.0
10 ---- 0.0


In [8]:
print('Valor Objetivo: ', modelo.obj())

Valor Objetivo:  309.0


### Pequenho problema de la mochila
#### V1

In [9]:
import pyomo.environ as pyEnv
path_solv = '../solvers/cbc.exe' 

In [10]:
#pesos = [10, 7, 8, 10, 4]
#valor = [6, 10, 6, 8, 0]
pesos = [7, 14, 7, 10, 4]
valor = [6, 10, 6, 8, 0]
capacidade = 16
indx = len(pesos)

In [11]:
model = pyEnv.ConcreteModel()

model.I = pyEnv.RangeSet(indx)
model.X = pyEnv.Var(model.I, within=pyEnv.Binary)
model.Obj = pyEnv.Objective(expr=sum(valor[i-1]*model.X[i] for i in model.I), sense=pyEnv.maximize)
model.Rest = pyEnv.Constraint(expr=sum(pesos[i-1]*model.X[i] for i in model.I) <= capacidade) 

In [12]:
solver = pyEnv.SolverFactory('cbc', executable=path_solv)
resul = solver.solve(model, tee=False)

In [13]:
list(model.I)
for i in list(model.X.keys()):
    print(i, "----",model.X[i]())

1 ---- 1.0
2 ---- 0.0
3 ---- 1.0
4 ---- 0.0
5 ---- 0.0


In [14]:
model.Obj()

12.0

#### V2

In [15]:
Lucros = [6,10,6,7,4,2,4,]
Volume = [10,5,7,5,6,2,1]
Pesos = [2,10,7,6,3,1,2]
Capacid_vol = 20 
Capacid_pes = 16
indice = len(Lucros)

In [16]:
model_2restr = pyEnv.ConcreteModel()

model_2restr.I = pyEnv.RangeSet(indice)
model_2restr.X = pyEnv.Var(model_2restr.I, within=pyEnv.Binary)
model_2restr.L = pyEnv.Param(model_2restr.I, initialize=lambda model, i: Lucros[i-1])
model_2restr.P = pyEnv.Param(model_2restr.I, initialize=lambda model, i: Pesos[i-1])
model_2restr.V = pyEnv.Param(model_2restr.I, initialize=lambda model, i: Volume[i-1])

def objt(model):
    #return sum(Lucros[i-1]*model.X[i] for i in model.I)
    return pyEnv.summation(model.L, model.X)

def rest1(model):
    return pyEnv.summation(model.P,model.X) <= Capacid_pes

def rest2(model):
    return pyEnv.summation(model.V,model.X) <= Capacid_vol


model_2restr.Obj = pyEnv.Objective(rule=objt, sense=pyEnv.maximize)
model_2restr.R1 = pyEnv.Constraint(rule=rest1)
model_2restr.R2 = pyEnv.Constraint(rule=rest2)

In [17]:
solver = pyEnv.SolverFactory('cbc', executable=path_solv)
resol = solver.solve(model_2restr, tee=False)

In [18]:
model_2restr.Obj()

22.0

In [19]:
for i in list(model_2restr.X.keys()):
    print(i,"---",model_2restr.X[i]())

1 --- 1.0
2 --- 1.0
3 --- 0.0
4 --- 0.0
5 --- 0.0
6 --- 1.0
7 --- 1.0


In [20]:
model.write('ProblemaMochila2Restricoes.lp', io_options={'symbolic_solver_labels': True})

('ProblemaMochila2Restricoes.lp', 2554467328976)

In [21]:
resol

{'Problem': [{'Name': 'unknown', 'Lower bound': 22.0, 'Upper bound': 22.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 7, 'Number of binary variables': 7, 'Number of integer variables': 7, 'Number of nonzeros': 7, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.03, 'Wallclock time': 0.03, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.23270535469055176}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}